# 분류
### 서비스 이탈예측 데이터

In [10]:
# 데이터 로드
import pandas as pd

x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv')

display(x_train.head())
display(y_train.head())
# y_train -> 고객 이탈 여부 (Exited) 0 or 1

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15799217,Zetticci,791,Germany,Female,35,7,52436.20,1,1,0,161051.75
1,15748986,Bischof,705,Germany,Male,42,8,166685.92,2,1,1,55313.51
2,15722004,Hsiung,543,France,Female,31,4,138317.94,1,0,0,61843.73
3,15780966,Pritchard,709,France,Female,32,2,0.00,2,0,0,109681.29
4,15636731,Ts'ai,714,Germany,Female,36,1,101609.01,2,1,1,447.73


,CustomerId,Exited
0,15799217,0
1,15748986,0
2,15722004,0
3,15780966,0
4,15636731,0


In [11]:
print(x_train.info())
print(x_train.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       6499 non-null   int64  
 1   Surname          6499 non-null   object 
 2   CreditScore      6499 non-null   int64  
 3   Geography        6499 non-null   object 
 4   Gender           6499 non-null   object 
 5   Age              6499 non-null   int64  
 6   Tenure           6499 non-null   int64  
 7   Balance          6499 non-null   float64
 8   NumOfProducts    6499 non-null   int64  
 9   HasCrCard        6499 non-null   int64  
 10  IsActiveMember   6499 non-null   int64  
 11  EstimatedSalary  6499 non-null   float64
dtypes: float64(2), int64(7), object(3)
memory usage: 609.4+ KB
None
CustomerId         6499
Surname            2289
CreditScore         459
Geography             3
Gender                4
Age                  69
Tenure               11
Balance     

In [12]:
# 불필요한 컬럼 제거
drop_col = ['CustomerId', 'Surname']
x_train_drop = x_train.drop(columns = drop_col)
x_test_drop = x_test.drop(columns = drop_col)


In [13]:
import sklearn
print(sklearn.__all__)
import sklearn.model_selection
print(dir(sklearn.model_selection))

['calibration', 'cluster', 'covariance', 'cross_decomposition', 'datasets', 'decomposition', 'dummy', 'ensemble', 'exceptions', 'experimental', 'externals', 'feature_extraction', 'feature_selection', 'gaussian_process', 'inspection', 'isotonic', 'kernel_approximation', 'kernel_ridge', 'linear_model', 'manifold', 'metrics', 'mixture', 'model_selection', 'multiclass', 'multioutput', 'naive_bayes', 'neighbors', 'neural_network', 'pipeline', 'preprocessing', 'random_projection', 'semi_supervised', 'svm', 'tree', 'discriminant_analysis', 'impute', 'compose', 'clone', 'get_config', 'set_config', 'config_context', 'show_versions']
['BaseCrossValidator', 'BaseShuffleSplit', 'FixedThresholdClassifier', 'GridSearchCV', 'GroupKFold', 'GroupShuffleSplit', 'KFold', 'LearningCurveDisplay', 'LeaveOneGroupOut', 'LeaveOneOut', 'LeavePGroupsOut', 'LeavePOut', 'ParameterGrid', 'ParameterSampler', 'PredefinedSplit', 'RandomizedSearchCV', 'RepeatedKFold', 'RepeatedStratifiedKFold', 'ShuffleSplit', 'Stratif

In [14]:
'''
pd.get_dummies
: 수치형 데이터로 변환 -> 가변수화  => 기계학습에 적합한 데이터 형태로 가공됨
- 수치형 데이터로만 변환하면 서로 간의 관계성이 생겨버림
(ex. 월요일=1, 화요일=2, 수요일=3 -> 1+2 = 3의 관계성 생김)
'''
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
x_train_dummies = pd.get_dummies(x_train_drop)
y = y_train['Exited']

# 원-핫 인코딩(범주형 변수 처리)
x_test_dummies = pd.get_dummies(x_test_drop)

# get_dummies 후 컬럼 수 / 순서 x_test를 x_train에 맞춰 정렬
# train과 컬럼 순서 동일하게 하기
# 더미화 하면서 순서대로 정렬을 이미 하기 때문에 오류가 난다면 해당 컬럼이 누락된 것일 수 o
x_test_dummies = x_test_dummies[x_train_dummies.columns]
print(help(train_test_split))

Help on function train_test_split in module sklearn.model_selection._split:

train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)
    Split arrays or matrices into random train and test subsets.
    
    Quick utility that wraps input validation,
    ``next(ShuffleSplit().split(X, y))``, and application to input data
    into a single call for splitting (and optionally subsampling) data into a
    one-liner.
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    *arrays : sequence of indexables with same length / shape[0]
        Allowed inputs are lists, numpy arrays, scipy-sparse
        matrices or pandas dataframes.
    
    test_size : float or int, default=None
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to

In [15]:
# 학습 데이터 분할 (훈련용 67%, 검증용 33%)
X_train, X_validation, Y_train, Y_validation = train_test_split(x_train_dummies, y, test_size=0.33, random_state=42)
# 모델 생성 및 학습
rf = RandomForestClassifier(random_state = 23)
# .fit()으로 훈련 수행 (입력: X_train , 정답: Y_train)
rf.fit(X_train, Y_train)

RandomForestClassifier(random_state=23)

In [20]:
import sklearn.metrics
print(dir(sklearn.metrics))

from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, precision_score
#model_score
# rf.predict(X_train) : 랜덤포레스트가 예측한 클래스(label) 반환
# 각 샘플에 대해 최종 다수결로 판단된 클래스 리턴
predict_train_label = rf.predict(X_train)
# rf.predict_proba(X_train): 각 클래스별 예측 확률을 반환 -> 클래스 1(Exited)일 확률
predict_train_proba = rf.predict_proba(X_train)[:,1]

predict_validation_label = rf.predict(X_validation)
predict_validation_prob = rf.predict_proba(X_validation)[:,1]

['ConfusionMatrixDisplay', 'DetCurveDisplay', 'DistanceMetric', 'PrecisionRecallDisplay', 'PredictionErrorDisplay', 'RocCurveDisplay', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_base', '_classification', '_dist_metrics', '_pairwise_distances_reduction', '_pairwise_fast', '_plot', '_ranking', '_regression', '_scorer', 'accuracy_score', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'auc', 'average_precision_score', 'balanced_accuracy_score', 'brier_score_loss', 'calinski_harabasz_score', 'check_scoring', 'class_likelihood_ratios', 'classification_report', 'cluster', 'cohen_kappa_score', 'completeness_score', 'confusion_matrix', 'consensus_score', 'coverage_error', 'd2_absolute_error_score', 'd2_log_loss_score', 'd2_pinball_score', 'd2_tweedie_score', 'davies_bouldin_score', 'dcg_score', 'det_curve', 'euclidean_distances', 'explained_variance_score', 'f1_score', 'fbeta_score', 'fowlkes_mallows_s

In [21]:
# 문제에서 묻는 것에 따라 모델 성능 확인하개
# 정확도(accuracy), f1_score, recall, precision -> model.predict로 결과 뽑기
# auc, 확률이라는 표현 있으면 model.predict_proba로 결과 뽑고 첫번째 행의 값을 가져오기
# model.predict_proba()[:,1]
print('train accuracy:', accuracy_score(Y_train, predict_train_label))
print('validation accuracy:', accuracy_score(Y_validation, predict_validation_label))
print('\n')
print('train f1_score:', f1_score(Y_train, predict_train_label))
print('validation accuracy:', f1_score(Y_validation, predict_validation_label))
print('\n')
print('train recall_score :', recall_score(Y_train, predict_train_label))
print('validation recall_score :', recall_score(Y_validation, predict_validation_label))
print('\n')
print('train precision_score :', precision_score(Y_train, predict_train_label))
print('validation precision_score :', precision_score(Y_validation, predict_validation_label))
print('\n')
# roc_auc_score:
# ROC곡선: 모델의 TPR(재현율, 민감도), FPR(오탐율)을 보여주는 그래프 (x축: FPR, y축: TPR)
# AUC: ROC 곡선 아래 면적 -> 1에 가까울 수록 더 좋은 분류기
print('train auc: ', roc_auc_score(Y_train, predict_train_proba))
print('validation auc: ', roc_auc_score(Y_validation, predict_validation_prob))

# test 데이터
predict_test_label = rf.predict(x_test_dummies)
predict_test_proba = rf.predict_proba(x_test_dummies)[:,1]

# accuracy, f1_score, recall, precision
# pd.DataFrame({'CustomerId': x_test.CustomerId, 'Exited': predict_test_label}).to_csv('00300000.csv', index=False)

# auc, 확률
# pd.DataFrame({'CustomerId': x_test.CustomerId, 'Exited': predict_test_proba}).to_csv('00300000.csv', index=False)

train accuracy: 1.0
validation accuracy: 0.8638694638694638


train f1_score: 1.0
validation accuracy: 0.5875706214689266


train recall_score : 1.0
validation recall_score : 0.45217391304347826


train precision_score : 1.0
validation precision_score : 0.8387096774193549


train auc:  1.0
validation auc:  0.8500703135079344
